# Llama Stack RAG Lifecycle

In this notebook, we will walk through the lifecycle of building and evaluating a RAG pipeline using Llama Stack. 

**Example: Torchtune Knowledge Agent** 

Throughout this notebook, we will build a knowledge agent that can answer questions about the Torchtune project. 

## 0. Setup

In [1]:
!pip install llama-stack-client==0.2.1 --quiet
!pip install llama-toolchain --quiet
!pip install llama-stack --quiet


[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from llama_stack_client import LlamaStackClient, Agent
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os
# try:
#     from google.colab import userdata
#     os.environ['FIREWORKS_API_KEY'] = userdata.get('FIREWORKS_API_KEY')
# except ImportError:
#     print("Not in Google Colab environment")

# client = LlamaStackAsLibraryClient("fireworks", provider_data = {"fireworks_api_key": os.environ['FIREWORKS_API_KEY']})
# _ = client.initialize()

# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

# MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"
# MODEL_ID = "mistral"
MODEL_ID="deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"

## 1. Simple Vanilla Agent

First, we will build a simple vanilla agent without any access to external knowledge base or tools, and check how it performs on a couple of questions. 


In [3]:
# First, let's come up with a couple of examples to test the agent
examples = [
    {
        "input_query": "What precision formats does torchtune support?",
        "expected_answer": "Torchtune supports two data types for precision: fp32 (full-precision) which uses 4 bytes per model and optimizer parameter, and bfloat16 (half-precision) which uses 2 bytes per model and optimizer parameter."
    },
    {
        "input_query": "What does DoRA stand for in torchtune?",
        "expected_answer": "Weight-Decomposed Low-Rank Adaptation"
    },
    {
        "input_query": "How does the CPUOffloadOptimizer reduce GPU memory usage?",
        "expected_answer": "The CPUOffloadOptimizer reduces GPU memory usage by keeping optimizer states on CPU and performing optimizer steps on CPU. It can also optionally offload gradients to CPU by using offload_gradients=True"
    },
    {
        "input_query": "How do I ensure only LoRA parameters are trainable when fine-tuning?",
        "expected_answer": "You can set only LoRA parameters to trainable using torchtune's utility functions: first fetch all LoRA parameters with lora_params = get_adapter_params(lora_model), then set them as trainable with set_trainable_params(lora_model, lora_params). The LoRA recipe handles this automatically."
    }
]

In [4]:
simple_agent = Agent(client,
                     model=MODEL_ID, 
                     instructions="You are a helpful assistant that can answer questions about the Torchtune project.")
for example in examples:
    simple_session_id = simple_agent.create_session(session_name=f"simple_session_{uuid.uuid4()}")
    response = simple_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=simple_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

Question: What precision formats does torchtune support?

Agent Answer: Okay, so I'm trying to figure out what precision formats Torchtune supports. I remember that 
precision in machine learning usually refers to the data type used to store numbers, which affects both memory 
usage and computation speed. Common formats I've heard about are float32, float16, and maybe even some others like 
bfloat16 or int8.

I think Torchtune is a library related to PyTorch, probably for model tuning or optimization. Since it's built on 
PyTorch, it might leverage PyTorch's native data types. I know PyTorch supports float32, float16, and bfloat16, 
especially on GPUs. Float16 uses less memory, which is good for large models, but it has less precision. Bfloat16 
is another 16-bit format that's used in some hardware, like Google's TPU.

Wait, does Torchtune support all these? Or maybe it has its own way of handling precision? I should check the 
documentation or the project's official sources. But since I don't have access to that right now, I'll have to rely
on what I know about PyTorch and similar libraries.

I also remember that some libraries support mixed precision training, where parts of the model use different 
precisions. For example, using float16 for computation and float32 for certain layers to maintain stability. 
Torchtune might support this kind of setup.

Another thought: sometimes libraries support quantization, which is a form of lower precision. Quantization can be 
done with 8-bit integers (int8) or other lower bit-width types. If Torchtune supports quantization, that would add 
more precision formats to its support list.

But I'm not sure if Torchtune specifically supports int8 or other quantized types. It might depend on the version 
or the specific features implemented. I should consider that Torchtune might rely on PyTorch's native types, so if 
PyTorch supports a certain precision, Torchtune likely does too.

In summary, I think Torchtune supports float32, float16, and bfloat16, possibly with mixed precision training. It 
might also support quantization types like int8 if that's part of PyTorch's capabilities. I should probably mention
that checking the official documentation would give the most accurate and up-to-date information.
</think>

The Torchtune project supports several precision formats, leveraging the capabilities of PyTorch. Here's a summary 
of the supported precision formats:

1. **Float32**: This is the standard 32-bit floating-point precision, widely used for its balance between precision
and memory usage.

2. **Float16**: A 16-bit floating-point format that uses half the memory of float32, making it suitable for large 
models. It offers lower precision but can be beneficial for training and inference speed.

3. **BFloat16**: Another 16-bit format that is supported by certain hardware, such as Google's TPU. It is useful 
for maintaining numerical stability in lower precision computations.

4. **Mixed Precision Training**: Torchtune likely supports configurations where different parts of the model use 
varying precisions, such as using float16 for computation and float32 for specific layers to maintain stability.

5. **Quantization**: Depending on the version and implementation, Torchtune might support quantization techniques, 
which use lower bit-width types like int8 to reduce memory usage and improve performance.

For the most accurate and up-to-date information, consulting the official Torchtune documentation is recommended.

Question: What does DoRA stand for in torchtune?

Agent Answer: Okay, so I'm trying to figure out what DoRA stands for in the context of the Torchtune project. I 
know that Torchtune is an open-source library for automated machine learning (AutoML) with a focus on deep 
learning, built on top of PyTorch. But I'm not exactly sure what DoRA refers to here.

First, I'll break down the term "DoRA." It could be an acronym, so I'll consider each letter: D, O, R, A. Maybe 
it's something like "Dynamic Resource Allocation" or "Distributed Optimization and Resource Allocation." But I'm 
not sure if that's accurate.

I remember that in AutoML, resource allocation is a key aspect, especially when dealing with computational 
resources like CPU and GPU. So DoRA might relate to how resources are managed during the training process. Maybe 
it's a feature in Torchtune that dynamically adjusts resources based on the model's needs.

I should check if there's any documentation or official sources from the Torchtune project that mention DoRA. If I 
look up Torchtune's GitHub repository or their documentation, I might find more information. Alternatively, 
searching for "DoRA Torchtune" could give me some clues.

Wait, I think I've heard the term DoRA in the context of distributed training and resource management. Maybe it's 
about dynamically allocating resources to different parts of the model or across different workers in a distributed
environment. That would make sense because efficient resource allocation can improve training efficiency and 
scalability.

Another angle: sometimes in AutoML, especially with hyperparameter tuning, the system needs to allocate resources 
(like compute time) to different trials. DoRA could be the mechanism that decides how much resources each trial 
gets based on their progress or potential.

I should also consider that DoRA might be a specific module or component within Torchtune. Perhaps it's part of the
tuning process where resources are dynamically adjusted to optimize the training process. For example, if a model 
isn't using all available GPUs efficiently, DoRA might redistribute resources to balance the load.

I'm trying to recall if I've seen any examples or tutorials where DoRA is mentioned. Maybe in a blog post or a 
paper about Torchtune. If I can't find exact information, I might have to make an educated guess based on common 
AutoML concepts.

Putting it all together, DoRA in Torchtune likely stands for Dynamic Resource Allocation. It probably refers to the
system's ability to dynamically adjust computational resources during the AutoML process to optimize training 
efficiency and resource utilization. This would help in scaling the AutoML process and making it more adaptable to 
different workloads and environments.

I think I'm on the right track, but I'm not 100% certain. To confirm, I should look for any official Torchtune 
documentation or releases that mention DoRA. If I can't find it, I might have to rely on the context and common 
AutoML practices to define it accurately.
</think>

DoRA in the context of the Torchtune project stands for Dynamic Resource Allocation. It refers to the system's 
capability to dynamically adjust computational resources during the automated machine learning (AutoML) process. 
This feature optimizes training efficiency and resource utilization, enhancing scalability and adaptability in 
different workloads and environments.

Question: How does the CPUOffloadOptimizer reduce GPU memory usage?

Agent Answer: Okay, so I'm trying to understand how the CPUOffloadOptimizer reduces GPU memory usage in the 
Torchtune project. I remember that in machine learning, especially with deep learning models, GPU memory is a 
critical resource because models can be really large. If the GPU runs out of memory, training can't proceed, which 
is a problem.

I think the CPUOffloadOptimizer must have something to do with moving some parts of the computation from the GPU to
the CPU. But how exactly does that help with memory? Well, if certain parts of the model or data are moved to the 
CPU, that should free up some GPU memory. But I'm not entirely sure about the specifics.

Let me break it down. When training a model, all the model parameters and the data batches are usually kept on the 
GPU because GPUs are faster for these operations. But if the model is too big, even keeping all the parameters on 
the GPU can cause it to run out of memory. So, maybe the CPUOffloadOptimizer moves some of these parameters to the 
CPU when needed.

I've heard of gradient checkpointing before. I think that's a technique where instead of keeping all the 
intermediate activations during the forward pass, you compute them on the fly and store just enough to compute the 
gradients. This reduces the memory needed during backpropagation. So, maybe the CPUOffloadOptimizer uses gradient 
checkpointing to save memory.

Another thing I remember is that some layers can be offloaded. If certain layers are moved to the CPU, their 
computations would happen on the CPU, which is slower but frees up GPU memory. So, the optimizer might identify 
which layers can be moved without causing too much slowdown and offload them.

I also think about mixed precision training. This uses lower-precision (like 16-bit) computations to save memory, 
but I'm not sure if that's part of the CPUOffloadOptimizer or a separate technique. Maybe the optimizer doesn't 
handle that directly but focuses more on moving data and computations.

Putting it all together, the CPUOffloadOptimizer probably works by moving parts of the model or computations to the
CPU when the GPU is running out of memory. It might use techniques like gradient checkpointing and layer offloading
to free up space. It's a strategy to balance computation speed and memory usage, ensuring that training can 
continue without running out of GPU memory.

I should also consider how this affects training time. Moving computations to the CPU is slower, so there's a 
trade-off between memory usage and speed. The optimizer likely tries to find the right balance to keep training 
efficient while preventing memory issues.

In summary, the CPUOffloadOptimizer reduces GPU memory by offloading some computations and data to the CPU, 
possibly using techniques like gradient checkpointing and layer offloading, thus allowing training to proceed 
without exceeding GPU memory limits.
</think>

The CPUOffloadOptimizer in the Torchtune project reduces GPU memory usage by strategically offloading certain 
computations and data to the CPU, thereby optimizing memory usage without significantly compromising training 
efficiency. Here's a detailed explanation of how it works:

1. **Computation Offloading**: The optimizer identifies parts of the model or computations that can be moved from 
the GPU to the CPU. This offloading frees up GPU memory, allowing training to proceed even with large models.

2. **Gradient Checkpointing**: This technique reduces memory usage during backpropagation by recomputing 
intermediate activations instead of storing them. This minimizes the memory footprint during the backward pass.

3. **Layer Offloading**: The optimizer may move specific layers to the CPU. While this slows down those 
computations, it effectively frees up GPU memory, enabling training to continue without memory issues.

4. **Balancing Efficiency and Memory**: The CPUOffloadOptimizer seeks a balance between computation speed and 
memory usage. It aims to k

Question: How do I ensure only LoRA parameters are trainable when fine-tuning?

Agent Answer: Okay, so I'm trying to figure out how to ensure that only the LoRA parameters are trainable when I'm 
fine-tuning a model using the Torchtune project. I remember that LoRA stands for Low-Rank Adaptation, and it's a 
method used to efficiently fine-tune large language models by modifying only a small number of parameters. This is 
supposed to make the fine-tuning process faster and less resource-intensive compared to fine-tuning the entire 
model.

First, I think I need to understand how LoRA works in the context of Torchtune. From what I recall, LoRA involves 
adding two small matrices to the attention layers of a transformer model. These matrices are typically low-rank, 
meaning they have a much smaller dimension than the original layers, which reduces the number of parameters that 
need to be updated during fine-tuning.

So, when using Torchtune, I believe there's a specific way to apply LoRA. Maybe it's through a decorator or a 
configuration setting. I think the Hugging Face library, which Torchtune is built on, has a LoRA approach, and 
perhaps Torchtune provides a similar or integrated method.

I remember seeing something about using the `LoraConfig` class in Hugging Face's `transformers` library. Maybe 
Torchtune has a similar configuration option. I should check if Torchtune has a `LoraConfig` or something 
equivalent. If it does, I can set that up to enable LoRA.

Once LoRA is enabled, I need to make sure that only the LoRA parameters are trainable. That means the rest of the 
model's parameters should be frozen or not updated during training. I think in PyTorch, you can set `requires_grad`
to False for the parameters you don't want to update. But with LoRA, perhaps Torchtune handles this automatically, 
so I just need to ensure that the LoRA parameters are the only ones with `requires_grad` set to True.

I also recall that when using LoRA, you typically use an adapter or a similar mechanism to apply the low-rank 
transformations. In Torchtune, maybe this is done through a specific trainer or a callback. I should look into the 
Torchtune documentation to see if there's a recommended way to apply LoRA and control which parameters are 
trainable.

Another thought: when initializing the model with LoRA, the base model's parameters are usually frozen. So, during 
the fine-tuning process, only the LoRA parameters are updated. I think this is done by setting the base model to 
evaluation mode or freezing its parameters. Maybe in Torchtune, this is handled by the `pl.Trainer` configuration 
or through some specific flags.

I should also consider the training loop. In PyTorch Lightning, which Torchtune uses, the training loop 
automatically updates the parameters based on the optimizer. So, if I set up the model correctly with only LoRA 
parameters trainable, the optimizer should only update those.

Wait, maybe I can use the `freeze` method provided by Torchtune or Hugging Face. For example, after applying LoRA, 
I can freeze the base model and unfreeze the LoRA layers. That way, during training, only the LoRA parameters are 
updated.

I'm a bit confused about how exactly to apply LoRA in Torchtune. Let me think step by step. First, I load the base 
model, then apply the LoRA adapter to it. Then, I need to make sure that the base model's parameters are not 
trainable. So, I can loop through the model's named parameters and set `requires_grad` to False for all except the 
LoRA ones.

Alternatively, maybe Torchtune provides a utility function to apply LoRA and freeze the base model automatically. I
should check the Torchtune documentation or examples to see if there's a standard way to do this.

Another point: when using LoRA, the model's forward pass is modified to include the LoRA transformations. So, I 
need to make sure that during the forward pass, the LoRA layers are active and their parameters are being used. I 
think this is handled by the LoRA implementation, but I should confirm.

I also wonder about the

#### 1.1 Evaluate Agent Responses
Let's gather up the agent's logs and evaluate the agent's performance. We can see that our agent's response is quite bad and off from the expected answer.

In [5]:
eval_rows = []
for i, session_id in enumerate(simple_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=simple_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

BadRequestError: Error code: 400 - {'detail': 'Invalid value: Pass OpenAI API Key in the header X-LlamaStack-Provider-Data as { "openai_api_key": <your api key>}'}

## 2. Search Agent

Now, let's see how we can improve the agent's performance by adding a search tool.

In [ ]:
search_agent = Agent(client, 
                     model=MODEL_ID,
                     instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the search tool to answer questions.",
                     tools=["builtin::websearch"])
for example in examples:
    search_session_id = search_agent.create_session(session_name=f"search_session_{uuid.uuid4()}")
    response = search_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=search_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

#### 2.1 Evaluate Agent Responses

We can see that with a search tool, the agent's performance is much better, and have less hallucinations. 

In [ ]:
eval_rows = []
for i, session_id in enumerate(search_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=search_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

### 3. RAG Agent

Now, let's see how we can improve the agent's performance by adding a RAG tool that explicitly retrieves information from Torchtune's documentation. 

In [ ]:
from llama_stack_client.types import Document
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "datasets.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]
documents = [
    Document(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

vector_providers = [
    provider for provider in client.providers.list() if provider.api == "vector_io"
]
selected_vector_provider = vector_providers[0]
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id=selected_vector_provider.provider_id,
)

client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

In [ ]:
rag_agent = Agent(
    client,
    model=MODEL_ID,
    instructions="You are a helpful assistant that can answer questions about the Torchtune project. You should always use the RAG tool to answer questions.",
    tools=[{
        "name": "builtin::rag",
        "args": {"vector_db_ids": [vector_db_id]},
    }],
)

for example in examples:
    rag_session_id = rag_agent.create_session(session_name=f"rag_session_{uuid.uuid4()}")
    response = rag_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=rag_session_id,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")

In [ ]:
eval_rows = []
for i, session_id in enumerate(rag_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=rag_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)

#### Deep dive into RAG Tool Performance
- Now, let's take a closer look at how the RAG tool is doing, specifically on the second example where the agent's answer is not correct on identifying what DoRA stands for. 
- Notice that the issue lies with the retrieval step, where the retrieved document is not relevant to the question. 

In [ ]:
session_response = client.agents.session.retrieve(agent_id=rag_agent.agent_id, session_id=rag_agent.sessions[1])
pprint(session_response.turns)

### 3.1 Improved RAG with Long Context

- Instead of performing reteival tool, we send documents as attachments to the agent and let it use the entire document context. 
- Note how that the model is able to understand the entire context from documentation and answers the question with better factuality with improved retrieval. 

In [ ]:
urls = [
    "memory_optimizations.rst",
    "chat.rst",
    "llama3.rst",
    "datasets.rst",
    "qat_finetune.rst",
    "lora_finetune.rst",
]

attachments = [
    {
        "content": {
            "uri": f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        },
        "mime_type": "text/plain",
    }

    for i, url in enumerate(urls)
]

rag_attachment_agent = Agent(
    client,
    model=MODEL_ID,
    instructions="You are a helpful assistant that can answer questions about the Torchtune project. Use context from attached documentation for Torchtune to answer questions.",
)

for example in examples:
    session_id = rag_attachment_agent.create_session(session_name=f"rag_attachment_session_{uuid.uuid4()}")
    response = rag_attachment_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": example["input_query"]
            }
        ],
        session_id=session_id,
        documents=attachments,
        stream=False
    )
    rich.print(f"[bold cyan]Question:[/bold cyan] {example['input_query']}")
    rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")



In [ ]:
eval_rows = []
for i, session_id in enumerate(rag_attachment_agent.sessions):
    session_response = client.agents.session.retrieve(agent_id=rag_attachment_agent.agent_id, session_id=session_id)
    for turn in session_response.turns:
        eval_rows.append({
            "input_query": examples[i]["input_query"],
            "expected_answer": examples[i]["expected_answer"],
            "generated_answer": turn.output_message.content,
        })

scoring_params = {
    "braintrust::factuality": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows,
    scoring_functions=scoring_params,
)
pprint(scoring_response)